In [1]:
# problema: Reducir  el tiempo de busqueda de un Robot a un item en un area que pertenece a una planta industrial.

In [2]:
# Agent
# Environment
# policy
# reward
# *render,  gym

import os
import random
from IPython.display import clear_output

"""
    0 1 2 3 4
    5 6 7 8 9
    ....
    .
"""

class Discrete:
    def __init__(self,num_actions: int):
        self.n = num_actions
    def sample(self):
        return random.randint(0, self.n-1)

        """
            0 1 2 3 4
            5 6 7 8 9
            ...
                23 24
        """

class Environment:
    # X,Y plane: UP, DOWN, RIGHT, LEFT
    seeker, goal = (0,0), (4,4)
    info = {'seeker': seeker, 'goal': goal}
    def __init__(self, *args, **kwargs):
        self.action_space = Discrete(4)
        self.observation_space = Discrete(25)
    
    def reset(self):
        self.seeker = (0,0)
        return self.get_observation()
    
    def get_observation(self):
        return 5*self.seeker[0]+self.seeker[1]
    
    def get_reward(self):
        return 1 if self.seeker == self.goal else 0
    
    def is_done(self):
        return self.seeker == self.goal
    
    def step(self, action):
        if action == 0:   # move down
            self.seeker = (min(self.seeker[0]+1,4), self.seeker[1])
        elif action == 1: # move left
            self.seeker = (self.seeker[0], max(self.seeker[1]-1,0))
        elif action == 2: # move up
            self.seeker = (max(self.seeker[0]-1,0), self.seeker[1])
        elif action == 3: # move right
            self.seeker = (self.seeker[0], min(self.seeker[1]+1,4))
        else:
            raise ValueError("Invalid action")
        return self.get_observation(), self.get_reward(), self.is_done(), self.info

    def render(self, *args, **kwargs):
        # os.system('cls' if os.name == 'nt' else 'clear')
        clear_output()
        grid = [['| ' for _ in range(5)] + ["|\n"] for _ in range(5)]
        grid[self.goal[0]][self.goal[1]] = '|G'
        grid[self.seeker[0]][self.seeker[1]] = '|S'
        print(''.join([''.join(grid_row) for grid_row in grid]))
        
import numpy as np
class Policy:
    def __init__(self, env):
        self.state_action_table = [[0 for _ in range(env.action_space.n)] for _ in range(env.observation_space.n)] # observation_space = 25
        self.action_space = env.action_space
        
    def get_action(self, state, explore=True, epsilon=0.1 ):
        if explore and (random.uniform(0,1) < epsilon):
            return self.action_space.sample()
        print("state:", state)
        print("table state:", self.state_action_table[state])
        print("argmax: ", np.argmax(self.state_action_table[state]))
        return np.argmax(self.state_action_table[state])
        


In [14]:
import time

class Simulation(object):
    def __init__(self, env) -> None:
        self.env = env

    def rollout(self, policy, render=False, explore=True, epsilon=0.1):
        experiences = []
        state = self.env.reset()
        done = False
        while not done:
            # random_action = env.action_space.sample()
            action = policy.get_action(state, explore, epsilon)
            next_state, reward, done, info = env.step(action)
            experiences.append([state, action, reward, next_state])    
            state = next_state
            if render:
                time.sleep(0.5)
                self.env.render()
        return experiences

def train_policy(env, num_episodes=10000):
    policy = Policy(env)
    sim = Simulation(env)
    for _ in range(num_episodes):
        experiences = sim.rollout(policy)
        update_policy(policy, experiences)
    return policy


def update_policy(policy, experiences):
    alpha=0.1
    discount_factor=0.9
    for state, action, reward, next_state in experiences:
        next_max = np.max(policy.state_action_table[next_state])
        value = policy.state_action_table[state][action]
        new_value = value*(1-alpha) + alpha*(reward+discount_factor*next_max)
        policy.state_action_table[state][action] = new_value

def evaluate_policy(env, policy, num_episodes=10):
    simulation = Simulation(env)
    steps = 0
    for _ in range(num_episodes):
        experiences = simulation.rollout(policy, render=True, explore=False)
        steps += len(experiences)
    
    print(f"{steps/num_episodes} steps on average "
            f" for a total of {num_episodes} episodes")


env = Environment()
opt_policy = train_policy(env)

evaluate_policy(env, opt_policy)


| | | | | |
| | | | | |
| | | | | |
| | | | | |
| | | | |S|

8.0 steps on average  for a total of 10 episodes


In [12]:
len(experiences)

398